In [32]:

import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import json
import numpy as np




In [33]:
# DATASET = "zero_shot_cot_50_50_merged"
# DATASET = "baseline_zero_shot_gpt3_50_50_merged"
DATASET = "auto_gpt3_gpt3_50_50_generations_merged"


GENERATIONS_PATH = "../datasets/error_analysis/" + DATASET + ".csv"
OUTPUT_PATH = "../datasets/error_analysis/structured_explanation_analysed/" + DATASET + ".csv"



data = pd.read_csv(GENERATIONS_PATH)
data.head()

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence,structured_explanation_v2,structured_explanation_v2_gpt4
0,0,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:55.032Z,2023-10-20T12:40:03.460Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",no match,False,True,FN,20%,90%,"Based on the information provided, the attribu...","Based on the information provided, the attribu..."
1,1,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:42.734Z,2023-10-20T12:39:53.607Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",The two products may match.,True,True,NONE,90%,80%,"[{""attribute"":""brand"",""importance"":""0.10"",""val...","[{""attribute"":""brand"",""importance"":""1.00"",""val..."
2,2,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:28.325Z,2023-10-20T12:39:40.801Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,90%,80%,NaN,"Based on the information provided, the attribu..."
3,3,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:10.835Z,2023-10-20T12:39:26.196Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,60%,80%,NaN,"Based on the information provided, here is a s..."
4,4,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:38:55.776Z,2023-10-20T12:39:08.718Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles of the two pr...",The two products may match.,True,True,NONE,90%,80%,NaN,"[{""attribute"":""brand"",""importance"":""0.5"",""valu..."


In [34]:
list = []

for i, row in enumerate(data["structured_explanation_v2_gpt4"]):
    try:
        row_json = json.loads(row)
        list.append(row_json)
    except Exception as e1:
        try:
            fixed = row.replace(",\n"+ row.split("\n")[-1], "]")
            row_json = json.loads(fixed)
            list.append(row_json)
            print("fixed:" + str(i))
        except Exception as e2:
            print("-----")
            print(e2)
            print(row)
            print(i)
            print("-----")


-----
Expecting value: line 1 column 1 (char 0)
Based on the information provided, the attributes can be broken down as follows:

[{"attribute":"brand","importance":"0.5","values":["Traveler's","Traveler's Company"]},
{"attribute":"product type","importance":"0.5","values":["Lined Notebook Refill","Notebook Accessories"]},
{"attribute":"size","importance":"0.3","values":["Regular","missing"]},
{"attribute":"line interval","importance":"0.2","values":["6.5mm","missing"]},
{"attribute":"description","importance":"0.5","values":["Specific","None"]}] 

Please note that the importance scores are subjective and can vary based on the specific requirements of the comparison.
0
-----
-----
Expecting value: line 1 column 1 (char 0)
Based on the information provided, the attributes can be broken down as follows:

[{"attribute":"brand","importance":"0.30","values":["Traveler's Company","missing"]},
{"attribute":"model","importance":"0.40", "values":["014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEB

In [35]:
len(list)

150

In [36]:

# Create empty dictionaries to store statistics
attribute_counts = {}
attribute_total_importance = {}
attribute_importances = {}
count = 0
# Iterate through the list of JSONs
for item in list:
    for item2 in item:
        count += 1
        attribute = item2['attribute']
        importance = float(item2['importance'])
        values = item2['values']

        # Update the dictionaries
        if attribute in attribute_counts:
            attribute_counts[attribute] += 1
            attribute_total_importance[attribute] += importance
            attribute_importances[attribute].append(importance)
        else:
            attribute_counts[attribute] = 1
            attribute_total_importance[attribute] = importance
            attribute_importances[attribute] = [importance]
print(count)
# Calculate average and variance
attribute_average_importance = {attribute: total_importance / attribute_counts[attribute] for attribute, total_importance in attribute_total_importance.items()}
attribute_variance_importance = {attribute: np.var(importances) for attribute, importances in attribute_importances.items()}



# Create a DataFrame from the calculated statistics
attribute_stats = pd.DataFrame({
    'attribute': attribute_counts.keys(),
    'count': attribute_counts.values(),
    'average importance': [attribute_average_importance[attr] for attr in attribute_counts.keys()],
    'variance of importance': [attribute_variance_importance[attr] for attr in attribute_counts.keys()]
})
# sum of counts
print(attribute_stats['count'].sum())
# Display the resulting table
attribute_stats



779
779


,attribute,count,average importance,variance of importance
0,brand,149,0.388926,0.094861
1,model,105,0.615238,0.102339
2,size,16,0.387500,0.107344
3,casing,6,0.450000,0.075000
4,compound,4,0.387500,0.097969
...,...,...,...,...
168,finish,1,0.250000,0.000000
169,grain quality,1,0.250000,0.000000
170,image formation,1,0.250000,0.000000
171,resistance to age-induced changes,1,0.250000,0.000000


In [37]:
# attribute_stats.to_csv(OUTPUT_PATH, index=False)